## Preprocess and explore datasets

## TODOs

- Try fb prophet for time series - only
- Explore spatial econometric methods
- Slice according to percentiles/quartiles to find which can give best results

In [2]:
import pandas as pd
import folium as fol
import geohash2 as gh
import seaborn as sns
import geopandas as gpd
from folium import plugins
import matplotlib.pyplot as plt
from shapely.geometry import Point
%matplotlib inline

In [3]:
ds_path = r'C:\Users\jose.luigi.s.torres\Desktop\aiforsea\traffic_management\data\training.csv'

df = pd.read_csv(ds_path)

In [4]:
######### TEST ON SUBSET FIRST #########
subset_test = pd.DataFrame(df.loc[:100,])
decoded = subset_test.geohash6.apply(lambda x: gh.decode_exactly(x))
subset_test['lat'] = decoded.apply(lambda x: x[0])
subset_test['long'] = decoded.apply(lambda x: x[1])

subset_test['hour'] = subset_test.timestamp.str.split(':').str[0]
subset_test['min'] = subset_test.timestamp.str.split(':').str[1]

geom_test = [Point(xy) for xy in zip(subset_test.lat.astype('float'),subset_test.long.astype('float'))]
gdf_test = gpd.GeoDataFrame(subset_test,geometry=geom_test)

subset_test.drop(['geohash6'],axis=1,inplace=True)

In [5]:
# preprocess coordinates
decoded = df.geohash6.apply(lambda x: gh.decode_exactly(x))
df['lat'] = decoded.apply(lambda x: x[0])
df['long'] = decoded.apply(lambda x: x[1])

# convert to geodataframe
geom = [Point(xy) for xy in zip(df.lat.astype('float'),df.long.astype('float'))]
gdf = gpd.GeoDataFrame(df,geometry=geom)

In [6]:
gdf['hour'] = gdf.timestamp.str.split(':').str[0]
gdf['min'] = gdf.timestamp.str.split(':').str[1]

gdf['hour'] = gdf['hour'].astype('int')
gdf['min'] = gdf['min'].astype('int')


gdf.drop(['geohash6',],axis=1,inplace=True)

In [16]:
map_subset = gdf.loc[gdf.demand>gdf.demand.quantile(0.7),].copy()

hm = fol.Map(map_subset[['lat','long']].sample().values,zoom_start=12)

map_subset.sort_values(['day','hour','min','lat','long'],inplace=True)

map_subset['t_lbl'] = map_subset.day.astype(str).str.cat(map_subset.timestamp,sep=" ")

# data = map_subset[['lat','long']].copy().values.tolist()
# time = map_subset.t_lbl.values.tolist()

df_hour_list = []
for lbl in map_subset['hour'].unique():
    df_hour_list.append(map_subset.loc[map_subset['hour'] == lbl, ['lat', 'long', 'demand']].groupby(['lat', 'long']).median().reset_index().values.tolist())


hm.add_child(plugins.HeatMapWithTime(df_hour_list))

In [12]:
map_subset = gdf.loc[gdf.demand>gdf.demand.quantile(0.7),].copy()

hm = fol.Map(map_subset[['lat','long']].sample().values,zoom_start=12)

map_subset.sort_values(['day','hour','min','lat','long'],inplace=True)

map_subset['t_lbl'] = map_subset.day.astype(str).str.cat(map_subset.timestamp,sep=" ")

# data = map_subset[['lat','long']].copy().values.tolist()
# time = map_subset.t_lbl.values.tolist()

df_hour_list = []
for lbl in map_subset['min'].unique():
    df_hour_list.append(map_subset.loc[map_subset['min'] == lbl, ['lat', 'long', 'demand']].groupby(['lat', 'long']).mean().reset_index().values.tolist())


hm.add_child(plugins.HeatMapWithTime(df_hour_list))